In [ ]:
import os
import sys
import snowflake.connector as sf_c
import pandas as pd
from datetime import datetime
import concurrent.futures
import boto3
import fnmatch
import zipfile
from io import BytesIO
import tarfile
import subprocess

s3_resource = boto3.resource('s3')
s3_client = boto3.client('s3')

CEDL_HOME=os.environ['CEDL_HOME']
nexus_connectionProfile = CEDL_HOME + '/etc/.sf.nexus.profile'
s3_connectionProfile = CEDL_HOME + '/etc/.s3_connection_profile'


def snowFlake_Connection():
    try:
        pathExist=os.path.exists(nexus_connectionProfile)
        if(not pathExist):
            print('The profile {} doesn''t exist'.format(nexus_connectionProfile))
            exit(1)
        profileFile=open(nexus_connectionProfile)
        for line in profileFile:
            if(line.split('=')[0]=='snowflakeAccount'):
                snowflakeAccount=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakeUsername'):
                snowflakeUsername=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakePassword'):
                snowflakePassword=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakeRole'):
                snowflakeRole=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakeDBName'):
                snowflakeDBName=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakeWarehouse'):
                snowflakeWarehouse=line.split('=')[1].replace('\n','')
            else:
                pass
        profileFile.close()
        if(len(snowflakeAccount)==0 or len(snowflakeUsername)==0 or len(snowflakePassword)==0 or len(snowflakeRole)==0 or len(snowflakeDBName)==0 or len(snowflakeWarehouse)==0):
            print('some parameters are missing from {}'.format(nexus_connectionProfile))
            exit(1)
        conn=sf_c.connect(user=snowflakeUsername, password=snowflakePassword, account=snowflakeAccount, warehouse=snowflakeWarehouse, database=snowflakeDBName)
        print("connected to SNOWFLAKE Database.")
    except sf_c.Error as e:
        print('Error connecting to SNOWFLAKE Database - {}'.format(e))
        exit(1)
    return conn



def get_job_details(conn, job_id):
    try:
        sf_cur = sf_conn.cursor()
        query = '''
        SELECT  processing_folder_name,
            is_src_file_dependent,
            is_src_file_compressed,
            compression_type,
            src_file_location,
            src_file_pattern,
            src_file_format,
            src_file_delimiter,
            outbound_extract_flag,
            outbound_extract_file_location,
            outbound_extract_file_name,
            is_pre_process_req,
            pre_process_scpt_type,
            archieve_file_location,
            is_archieve_file_to_be_deleted,
            days_to_be_archieved,
            no_of_parallel_threads,
            pre_process_scpt_args
        FROM NEXUS.JMAN.JOB
        WHERE JOB_ID = %s
        '''

        sf_cur.execute(query, (job_id,))
        result = sf_cur.fetchone()

        if sf_cur.rowcount == 0:
            print('JOB details for jobid ({}) is not present in JMAN.JOB table'.format(job_id))
            exit(1)
        elif sf_cur.rowcount > 1:
            print('Duplicate jobid {} in JMAN.JOB table'.format(job_id))
            exit(1)
        else:
            processing_folder_name, is_src_file_dependent, is_src_file_compressed, compression_type,  src_file_location, src_file_pattern,  src_file_format, src_file_delimiter, outbound_extract_flag, outbound_extract_file_location, outbound_extract_file_name, is_pre_process_req, pre_process_scpt_type, archieve_file_location, is_archieve_file_to_be_deleted, days_to_be_archieved, no_of_parallel_threads, pre_process_scpt_args  = result
    except Exception as e:
        print('Error getting job details: {}'.format(e))
        exit(1)
    return  processing_folder_name, is_src_file_dependent, is_src_file_compressed, compression_type,  src_file_location, src_file_pattern,  src_file_format, src_file_delimiter, outbound_extract_flag, outbound_extract_file_location, outbound_extract_file_name, is_pre_process_req, pre_process_scpt_type, archieve_file_location, is_archieve_file_to_be_deleted, days_to_be_archieved, no_of_parallel_threads, pre_process_scpt_args



sf_conn = snowFlake_Connection()
param_value_example = 'LOAD_PARACHUTE_CHHA'


result = get_job_details(sf_conn,param_value_example)